In [18]:
import subprocess
import math
import os
from pydub import AudioSegment


# video_path: 동영상의 경로
# audio_path: 추출한 audio가 저장될 경로
def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-i",  # input
        video_path,
        "-vn",  # video nope: ignore the video
        audio_path,
        "-y",  # 이미 파일이 존재하는 경우 덮어쓰기를 함
    ]
    # subprocess는 python code에서 command line 명령을 사용할 수 있게 해줌
    subprocess.run(command)


# 확장 가능한 함수로 작성
# audio_path: 오디오트랙 파일이 있는 곳, chunk_size: 몇 분 단위로 쪼갤지, chunks_folder: 쪼갠 오디오트랙을 저장할 폴더
def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000
    chunks = math.ceil(len(track) / chunk_len)
    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len

        chunk = track[start_time:end_time]
        chunk_path = f"{chunks_folder}/chunk_{i:03d}.mp3"
        directory = os.path.dirname(chunk_path)

        if not os.path.exists(directory):
            os.makedirs(directory, exist_ok=True)

        chunk.export(chunk_path, format="mp3")

In [3]:
cut_audio_in_chunks("./files/podcast.mp3", 10, "./files/chunks")

In [19]:
from openai import OpenAI

# glob는 패턴을 사용하여 디렉토리 내부의 파일을 검색할 수 있도록 해줌
import glob

client = OpenAI()


def transcribe_chunks(chunk_folder, destination):
    files = sorted(glob.glob(f"{chunk_folder}/*.mp3"))

    for file in files:
        # "a": appending mode로 파일을 열었기 때문에 기존 파일의 내용을 덮어쓰는 것이 아니라 기존 내용 뒤에 붙여나감
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
            )
            text_file.write(transcript.text)

In [20]:
transcribe_chunks("./files/chunks/", "./files/transcript.txt")